In [23]:
using Plots, Roots
plotlyjs()

INFO: Precompiling module Roots.


Plots.PlotlyJSBackend()

# CircleConv.m

In [165]:
function CircleConv(fltr, inputs)
    # Convolves fltr with inputs and removes extraneous values
    #fltr is assumed to have an odd number of elements and be centered
    #Replicates inputs for periodic boundary conditions
    
    x = conv(fltr, [inputs; inputs; inputs])
    extra = ceil(length(fltr)/2)
    start = Int64(1 + extra)
    finish = Int64(length(x) - extra)
    x = x[start:finish]
    Result = x[length(inputs) + 1 : 2*length(inputs)]
    return Result
end

CircleConv (generic function with 1 method)

# Hallucinations.m
### Requires gif

In [9]:
Size = 64;  #Spatial size of array
EE = zeros(Size, Size)
IN = zeros(Size, Size)
Stim = zeros(Size, Size)
QStim = zeros(Size, Size)
DX = 20; #microns
Last = 150;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)
X = DX*(1:Size)
DT = 10
DelT = 1
[Xsyn, Ysyn] = meshgrid(-Size/2:(Size/2 - 1))
Radius = DX*sqrt(Xsyn.^2 + Ysyn.^2)
Mask = (abs(Xsyn) <= 30).*(abs(Ysyn) <= 30); #for boundary conditions

#**********
EEgain = 1.95/40
EIgain = 1.4/60
IIgain = 2.2/30
StimTime = 5
Q = 2.0
Bias = input("Please choose initial bias (1) Vertical, (2) Horizontal, (3) Oblique, (4) Checks: ")
if Bias == 1; Stim = 3*cos(2*pi*Xsyn*3/Size); end
if Bias == 2; Stim = 3*cos(2*pi*Ysyn*3/Size); end
if Bias == 3; Stim = 3*cos(2*pi*(Xsyn + Ysyn)*2/Size); end
if Bias == 4; Stim = 3*cos(2*pi*Xsyn*2/Size).*cos(2*pi*Ysyn*2/Size); end
Stim = Stim + 3*(rand(Size, Size) > 0.8)
#**********

synEE = EEgain*exp(-abs(Radius)/40)
synEE = fft2(synEE)
synEI = EIgain*exp(-abs(Radius)/60)
synEI = fft2(synEI)
synII = IIgain*exp(-abs(Radius)/30)
synII = fft2(synII)
whitebg('w')

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
        P = Stim*(T <= StimTime) + 1.8
        QS = QStim*(T <= StimTime) + Q
        Efft = fft2(EE)
        Ifft = fft2(IN)
        EEresp = real(ifft2(synEE.*Efft - synEI.*Ifft))
        EEresp = fftshift(EEresp) + P
        EEresp = (EEresp.*(EEresp > 0)).^2
        INresp = real(ifft2(synEI.*Efft - synII.*Ifft))
        INresp = fftshift(INresp) + QS
        INresp = (INresp.*(INresp > 0)).^2;     
        EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
        IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
        EE = 0.5*EE + 0.5*EE.*Mask
        EC[T] = EE[Size/2]
        IC[T] = IN[Size/2]
        if rem(T, 2) == 0
                figure(1); PS = image(EE); colormap(hot[64]); axis square
                pause(0.2)
        end
end

LoadError: [91msyntax: invalid assignment location "[Xsyn,Ysyn]"[39m

# Motion_Demo.m
### Requires gif

In [ ]:
#Transparent & Coherent motion demo()
Slope = input("Directions (1) ±22¡ or (2) ±68¡: ")
if Slope == 1; SX = 1; SY = 2.5; else SX = 2.5; SY = 1; end
[X, Y] = meshgrid(-100:100)
Apertures = ((X + 25).^2 + (Y + 25).^2) < 20^2
Apertures = Apertures + (((X - 25).^2 + (Y + 25).^2) < 20^2)
Apertures = Apertures + (((X + 25).^2 + (Y - 25).^2) < 20^2)
Apertures = Apertures + (((X - 25).^2 + (Y - 25).^2) < 20^2)
Lines = 100*cos(2*pi*(X + Y)/25)
figure(1), image(Lines); axis square; axis off; colormap(gray[200])
Motion = moviein(40)
for T = 1:40;  #amination
	Lines = 100*cos(2*pi*(-SX*X.*sign(X).*sign(Y) + SY*Y + 2*T)/40)
	Lines = Lines.*(Apertures) + 100
	Lines[80:120, 10:50] = Lines[55:95, 55:95]
	Lines[10:50, 80:120] = Lines[55:95, 105:145]
	Lines[80:120, 150:190] = Lines[55:95, 105:145]
	Lines[150:190, 80:120] = Lines[55:95, 55:95]
	Lines[20:60, 20:60] = Lines[55:95, 105:145]
	Lines[140:180, 140:180] = Lines[55:95, 105:145]
	Lines[20:60, 140:180] = Lines[55:95, 55:95]
	Lines[140:180, 20:60] = Lines[55:95, 55:95]
	image(Lines); axis square; axis off; colormap(gray[200])
	Motion[:, T] = getframe()
end
movie(Motion, 5)


# NeuralConv.m

In [252]:
function NeuralConv(fltr, inputs)
    # Convolves fltr with inputs and removes extraneous values
    #fltr is assumed to have an odd number of elements and be centered
    #Replicates inputs for periodic boundary conditions
    Sz = length(inputs)
    Xx = conv(fltr, inputs)
    extra = (length(fltr)/2)
    Xx = Xx[1 + extra: length(Xx) - extra]
    Result = Xx
    return Result
end

NeuralConv (generic function with 1 method)

# ParabolaInt.m

In [253]:
function ParabolaInt(Rminus1, Rmax, Rplus1, MaxX, Del)
    #Function performs parabolic interpolation on input()s
    Num = Del*(Rplus1 - Rminus1)
    Denom = 2.0*(Rplus1 + Rminus1 - 2.0*Rmax)
    PB = -Num./Denom - MaxX
    return PB
end


ParabolaInt (generic function with 1 method)

# Retina.m
### Need to figure out roots in Julia

In [254]:
function Retina()
    Total_Equations = 5;  #Solve for this number of interacting Neurons
    DT = 2;  #Time increment as fraction of time constant
    Final_Time = 1000;   #Final time value for calculation
    Last = Int64(Final_Time/DT + 1);  #Last time step
    Time = DT*(0:Last-1);  #Time vector
    WTS = [1 2 2 1];  #Runge-Kutta Coefficient weights

    X = Array{Float64}(Total_Equations, Last)
    K = Array{Float64}(Total_Equations, 4)
    Weights = Array{Float64}(Total_Equations, 4)


    for NU = 1:Total_Equations;  #Initialize
        X[NU, :] = zeros(1, Last);  #Vector to store response of Neuron #1
        K[NU, :] = zeros(1, 4);  #Runge-Kutta terms	
        Weights[NU, :] = WTS;  #Make into matrix for efficiency in main loop
    end
    Wt2 = [0 .5 .5 1];  #Second set of RK weights
    rkIndex = [1 1 2 3]

    L = 10^3#input("Light intensity (0 < L <= 10^4): ")
    DelL = 2#input("Test flash increment or decrment (Delta): ")
    a = 9
    b = 91
    c = 10
    d = -18*L
    quart(x) = a*x^3 + b*x^3 + c*x + d

    ### HARD CODED FOR NOW###
    Rts = [-8.4068; -4.5862; 2.8819] #Rts = roots([9 91 10 -10*L])

    X[3, 1] = Rts[3]; #Bipolar and other response levels in steady state
    X[4, 1] = X[3, 1]
    G = X[4, 1]/10
    X[1, 1] = X[3, 1]*(1 + 9*X[3, 1])
    X[2, 1] = X[1, 1]
    X[5, 1] = 50*X[3, 1]/(13 + X[3, 1])

    for T = 2:Last
      for rk = 1:4  #Fourth Order Runge-Kutta
        XH = X[:, T-1] + K[:, rkIndex[rk]]*Wt2[rk]
        Tme =Time[T-1] + Wt2[rk]*DT;  #Time upgrade

        LL = L + DelL*(Tme > 100)
        K[1, rk] = (DT/10)*(-XH[1] + LL - G*XH[2]);	#Your Equation Here
        K[2, rk] = (DT/100)*(-XH[2] + XH[1]);  #Your Equation Here
        K[3, rk] = (DT/10)*(-XH[3] + (6*XH[1] - 5*XH[2])*(6*XH[1] > 5*XH[2])/(1 + 9*XH[4]))
        K[4, rk] = (DT/80)*(-XH[4] + XH[3])
        K[5, rk] = (DT/10)*(-XH[5] + 50*XH[3]/(13 + XH[3]))

     end
        X[:, T] = X[:, T-1] + sum((Weights.*K)', 1)'/6
    end

    trace1 = PlotlyJS.scatter(;x=Time, y=X[5, :], mode="lines", line_color="red")
    layout = PlotlyJS.Layout(title="Solution", xlabel="Time (ms)", ylabel="Ganglion Cell Spike Rate (Hz)")
    p1 = PlotlyJS.plot([trace1], layout)

    # figure(1); Za = plot(Time, X[5, :], "-r'); set(Za, 'LineWidth", 2)
    # xlabel("Time (ms)'); ylabel('Ganglion Cell Spike Rate (Hz)")

    Increment = maximum(X[5, :]) - X[5, 2]
    Steady_State = X[5, Last]

    p1
end

Retina (generic function with 1 method)

In [255]:
Retina()

# VectorWTA.m
### Perceived length is still wrong

In [256]:
# fix.m
function fix(x)
   if x < 0
        return ceil
end

LoadError: [91msyntax: incomplete: "function" at In[256]:2 requires end[39m

In [258]:
# function VectorWTA()
Neurons = zeros(24)
Directions = (-180:15:165)*pi/180; #Preferred directions
NV = 2

#**********
#NV = input("Number of input vectors to network (<= 4) = ")
#**********

Tau = 10
Size = zeros(1, NV)
Ang = zeros(1, NV)
for K = 1:NV
    Number = K
    Size[K] = 2#input("Length of Vector (>= 1) = ")
    Ang[K] = 45#input("Vector Angle in deg re. Horizontal = ")
end

Ang = Ang*pi/180; #convert to radians
Inputs = zeros(1, 24)
for K = 1:NV
    global Inputs
    global K
    Inputs = Inputs + Size[K]*cos.(Directions'- Ang[K]).*(cos.(Directions' - Ang[K]) .>= 0)
end

#     figure(1), IP = polar(Directions, Inputs); set(IP, "LineWidth", 2)


Resp = zeros(1, 100)
Isynapses = vec(-3.*[1 1 1 1 1 1 0 0 0 0 0 1 1 1 1 1 1]); #Spread of feedback inhibition
for T = 2:1:100

    PSP = Inputs[:] + CircleConv(Isynapses, Neurons)
    PSP = (PSP ).*(PSP .> 0); #Threshold of 0
    Neurons = Neurons + (2/Tau)*(-Neurons + PSP)
    Resp[T] = Neurons[13]
end

#     figure(2), ZB = polar(Directions, Neurons, "r-'); set(ZB, 'LineWidth", 2)
#     figure(3), TB = plot(1:100, Resp, "r-'); set(TB, 'LineWidth", 2)

#These lines compute the perceived output direction with Parabolic interpolation
MAX = maximum(Neurons)
jj = find(Neurons.==MAX)

plotorient = -(-180:15:165)

PerceivedDirection = ParabolaInt(Neurons[jj - 1], Neurons[jj], Neurons[jj+1], plotorient[jj], 15.0)
TrueVectorDirection = atan2(sum(Size.*sin.(Ang)), sum(Size.*cos.(Ang)))*180/pi
DirectionError = TrueVectorDirection - PerceivedDirection
PerceivedLength = MAX
TrueVectorLength = sum(Size.*cos.(Ang - TrueVectorDirection*pi/180))
# end

4.0

# WCcortexAT.m
### Requires gif

In [259]:

#Wilson-Cowan Equations

Size = 100;  #Spatial size of array
EE = zeros(1, Size)
IN = zeros(1, Size)
Stim = zeros(1, Size)
Width = 5
Width = round(Width/2)
Stim[Size/2 - Width:Size/2 + Width] = 1.4*ones(1, 2*Width + 1)
Last = 50;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)
DX = 20; #microns
X = DX*(1:Size)
DT = 10
DelT = 0.5
Xsyn = DX*(-15:15)

#**********
EEgain = 1.5
EIgain = 1.3
IIgain = 1.5
StimTime = 5
Q = 0
#**********

synEE = EEgain*exp(-abs(Xsyn)/40)
synEI = EIgain*exp(-abs(Xsyn)/60)
synII = IIgain*exp(-abs(Xsyn)/30)

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
	P = Stim*(T <= StimTime)
	EEresp = NeuralConv[synEE, EE] - NeuralConv[synEI, IN] + P
	EEresp = (EEresp.*(EEresp > 0)).^2
	INresp = NeuralConv[synEI, EE] - NeuralConv[synII, IN] + Q
	INresp = (INresp.*(INresp > 0)).^2
	EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
	IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
	EC[round(T)] = EE[Size/2]
	IC[round(T)] = IN[Size/2]
	if rem(T, 2) == 0
		figure(1); PS = plot(X, EE, "r-', X, IN, 'b-'); set(PS, 'LineWidth", 2)
		axis([0 DX*Size 0 30])
		xlabel("Distance in microns'); ylabel('E (red) & I (blue) Responses")
		pause(0.2)
	end
end

Time = 1:Last
ST = (Time <= StimTime)
figure(2); PT = plot(Time, EC, "-r', Time, IC, 'b-', Time, ST, 'k-'); set(PT, 'LineWidth", 2)
xlabel("Time (ms)'); ylabel('E & I Responses")


LoadError: [91mArgumentError: invalid index: 48.0[39m

# WCcortexOSC.m
### Requires gif

In [260]:

Size = 100;  #Spatial size of array
EE = zeros(1, Size)
IN = zeros(1, Size)
Stim = zeros(1, Size)
Width = 800#input("Width of stimulus in microns (99 < Width < 1600) = ")
Width = round(Width/(2*DX))
Stim[Int64(Size/2 - Width):Int64(Size/2 + Width)] = 2*ones(1, 2*Width + 1)
Last = 300;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)
DX = 20; #microns
X = DX*(1:Size)
DT = 10
DelT = 0.5
Xsyn = DX*(-15:15)



#**********
EEgain = 1.9
EIgain = 1.5
IIgain = 1.5
Q = 0
#**********

synEE = EEgain*exp.(-abs(Xsyn)/40)
synEI = EIgain*exp.(-abs(Xsyn)/60)
synII = IIgain*exp.(-abs(Xsyn)/30)

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
	P = Stim
	EEresp = NeuralConv(vec(synEE), vec(EE)) - NeuralConv(vec(synEI), vec(IN)) + P
	EEresp = (EEresp.*(EEresp > 0)).^2
	INresp = NeuralConv(synEI, EE) - NeuralConv(synII, IN) + Q
	INresp = (INresp.*(INresp > 0)).^2
	EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
	IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
	EC[round(T)] = EE[Size/2]
	IC[round(T)] = IN[Size/2]
	if rem(T, 4) == 0
		figure(1); PS = plot(X, EE, "r-', X, IN, 'b-'); set(PS, 'LineWidth", 2)
		axis([0 DX*Size 0 100])
		xlabel("Distance in microns'); ylabel('E (red) & I (blue) Responses")
		pause(0.2)
	end
end

Time = 1:Last
figure(2); PT = plot(Time, EC, "-r', Time, IC, '-b'); set(PT, 'LineWidth", 2)
xlabel("Time (ms)'); ylabel('E (red) & I (blue) Responses")


Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::StepRange{Int64,Int64}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/derekfulton/.julia/v0.6/IJulia/src/execute_request.jl:160
 [5] eventloop(::ZMQ.Socket) at /Users/derekfulton/.julia/v0.6/IJulia/src/eventloop.jl:8
 [6] (::IJulia.##11#14)() at ./task.jl:335
while loading In[260], in expression starting on line 27
Stacktrace:
 [1] depwarn(::String, ::Symbol) at ./deprecated.jl:70
 [2] abs(::StepRange{Int64,Int64}) at ./deprecated.jl:57
 [3] include_string(::String, ::String) at ./loading.jl:515
 [4] execute_request(::ZMQ.Socket, ::IJulia.Msg) at /Users/derekfulton/.julia/v0.6/IJulia/src/execute_request.jl:160
 [5] eventloop(::ZMQ.Socket) at /Users/derekfulton/.julia/v0.6/IJulia/src/eventloop.jl:8
 [6] (::IJulia.##11#14)() at ./task.jl:335
while loading In[260], in expression starting on line 28
Stacktrace:
 [1] dep

LoadError: [91mArgumentError: invalid index: 16.5[39m

# WCcortexSTM.m
### Requires GIF

In [261]:
Size = 100;  #Spatial size of array
EE = zeros(1, Size)
IN = zeros(1, Size)
DX = 20; #microns
X = DX*(1:Size)
DT = 10
DelT = 0.5
Xsyn = DX*(-15:15)
Stim = zeros(1, Size)
Width = input("Width of stimulus in microns (99 < Width < 1600) = ")
Width = round(Width/(2*DX))
Stim[Size/2 - Width:Size/2 + Width] = ones(1, 2*Width + 1)
Last =  80;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)

#**********
EEgain = 1.95
EIgain = 1.4
IIgain = 2.2
StimTime = 10
Q = 0
#**********

synEE = EEgain*exp(-abs(Xsyn)/40)
synEI = EIgain*exp(-abs(Xsyn)/60)
synII = IIgain*exp(-abs(Xsyn)/30)

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
	P = Stim*(T <= StimTime)
	EEresp = NeuralConv[synEE, EE] - NeuralConv[synEI, IN] + P
	EEresp = (EEresp.*(EEresp > 0)).^2
	INresp = NeuralConv[synEI, EE] - NeuralConv[synII, IN] + Q
	INresp = (INresp.*(INresp > 0)).^2
	EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
	IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
	EC[round(T)] = EE[round(Size/2)]
	IC[round(T)] = IN[round(Size/2)]
	if rem(T, 4) == 0
		figure(1); PS = plot(X, EE, "r-', X, IN, 'b-'); set(PS, 'LineWidth", 2)
		axis([0 DX*Size 0 110])
		xlabel("Distance in microns'); ylabel('E (red) & I (blue) Responses")
		pause(0.2)
	end
end

Time = 1:Last
ST = (Time <= StimTime)
figure(2); PT = plot(Time, EC, "-r', Time, IC, 'b-', Time, ST, 'k-'); set(PT, 'LineWidth", 2)
xlabel("Time (ms)'); ylabel('E (red) & I (blue) Responses")


LoadError: [91mUndefVarError: input not defined[39m

# WCCortexWAVES.m
### Requires gif

In [263]:
Size = 400;  #Spatial size of array
EE = zeros(1, Size)
IN = zeros(1, Size)
Stim = zeros(1, Size)
Width = 100
DX = 20; #microns
Width = round(Width/(2*DX))
Last = 80;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)
X = DX*(1:Size)
DT = 10
DelT = 0.5
Xsyn = DX*(-15:15)

#**********
EEgain = 1.9
EIgain = 1.5
IIgain = 1.5
StimTime = 5
Q = -90

#To observe wave anniahilation, comment out first line and activate other two below
Stim[Size/2 - Width:Size/2 + Width] = 2*ones(1, 2*Width + 1)
#Stim[61:70] = 4*ones(1, 10)
#Stim[Size - 69:Size-60] = 4*ones(1, 10)
#**********

synEE = EEgain*exp(-abs(Xsyn)/40)
synEI = EIgain*exp(-abs(Xsyn)/60)
synII = IIgain*exp(-abs(Xsyn)/30)

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
	P = Stim*(T <= StimTime)
	EEresp = CircleConv[synEE, EE] - CircleConv[synEI, IN] + P
	EEresp = (EEresp.*(EEresp > 0)).^2
	INresp = CircleConv[synEI, EE] - CircleConv[synII, IN] + Q
	INresp = (INresp.*(INresp > 0)).^2
	EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
	IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
	EC[round(T)] = EE[Size/2]
	IC[round(T)] = IN[Size/2]
	if rem(T, 2) == 0
		figure(1); PS = plot(X, EE, "r-', X, IN, 'b-'); set(PS, 'LineWidth", 2)
		axis([0 DX*Size 0 100])
		xlabel("Distance in microns'); ylabel('E (red) & I (blue) Responses")
		pause(0.2)
	end
end

LoadError: [91mArgumentError: invalid index: 198.0[39m

# WCstability.m
### Requires gif

In [264]:
Size = 200;  #Spatial size of array
EE = 0.25*ones(1, Size)
IN = 3.85*ones(1, Size)
DX = 20; #microns
X = DX*(1:Size)
DT = 10
DelT = 0.5
Xsyn = DX*(-15:15)
Stim = ones(1, Size)
Last =  300;  #last time in computation
EC = zeros(1, Last)
IC = zeros(1, Last)
freq = input("Spatial Frequency (integer from 0-50) for stimulus = ")

#**********
EEgain = 1.95
EIgain = 1.4
IIgain = 2.2
freq = freq/maximum(X)
#**********

synEE = EEgain*exp(-abs(Xsyn)/40)
synEI = EIgain*exp(-abs(Xsyn)/60)
synII = IIgain*exp(-abs(Xsyn)/30)

for T = 1:DelT:Last;  #Loop in ms, Euler solution method
	P = 31.5*Stim + 0.01*cos(2*pi*freq*X)*(T <= 5)
	Q = 32.3*Stim
	EEresp = CircleConv[synEE, EE] - CircleConv[synEI, IN] + P
	EEresp = (EEresp.*(EEresp > 0)).^2
	INresp = CircleConv[synEI, EE] - CircleConv[synII, IN] + Q
	INresp = (INresp.*(INresp > 0)).^2
	EE = EE + (DelT/DT)*(-EE + 100*EEresp./(20^2 + EEresp))
	IN = IN + (DelT/DT)*(-IN + 100*INresp./(40^2 + INresp))
	EC[round(T)] = EE[Size/2]
	IC[round(T)] = IN[Size/2]
	if rem(T, 5) == 0
		figure(1); PS = plot(X, EE, "r-', X, IN, 'b-'); set(PS, 'LineWidth", 2)
		axis([0 DX*Size 0 110])
		xlabel("Distance in microns'); ylabel('E (red) & I (blue) Responses")
		pause(0.1)
	end
end
Time = 1:Last


LoadError: [91mUndefVarError: input not defined[39m

# WTA5Neurons.m

In [273]:
Total_Neurons = 5  #Solve for this number of interacting Neurons
DT = 2  #Time increment as fraction of time constant
Final_Time = 1000   #Final time value for calculation
Last = Int64(Final_Time/DT + 1)  #Last time step
Time = DT*(0:Last-1)  #Time vector
Tau = 20  #Neural time constants in msec
WTS = [1 2 2 1]  #Runge-Kutta Coefficient weights


X = Array{Float64}(Total_Equations, Last)
K = Array{Float64}(Total_Equations, 4)
Weights = Array{Float64}(Total_Equations, 4)


for NU = 1:Total_Neurons  #Initialize
	X[NU, :] = zeros(1, Last)  #Vector to store response of Neuron #1
	K[NU, :] = zeros(1, 4)  #Runge-Kutta terms	
	X[1, 1] = 0  #Initial conditions here if different from zero
	X[2, 1] = 0  #Initial conditions here if different from zero
	Weights[NU, :] = WTS  #Make into matrix for efficiency in main loop
end

Wt2 = [0 .5 .5 1]  #Second set of RK weights
rkIndex = [1 1 2 3]

#**********
Stim1 = 80
Stim2 = 79.8
Stim3 = 79.8
Stim4 = 79.8
Stim5 = 79.8
#**********


for T = 2:Last
  for rk = 1:4  #Fourth Order Runge-Kutta
	XH = X[:, T-1] + K[:, rkIndex[rk]]*Wt2[rk]
	Tme =Time[T-1] + Wt2[rk]*DT;  #Time upgrade
	
	Inh = sum(XH) - XH;  #Inhibitory strength	
	PSP1 = (Stim1 - 3*Inh[1])*(Stim1 > 3*Inh[1]);	
	PSP2 = (Stim2 - 3*Inh[2])*(Stim2 > 3*Inh[2]);		
	PSP3 = (Stim3 - 3*Inh[3])*(Stim3 > 3*Inh[3]);	
	PSP4 = (Stim4 - 3*Inh[4])*(Stim4 > 3*Inh[4]);		
	PSP5 = (Stim5 - 3*Inh[5])*(Stim5 > 3*Inh[5]);			
 	K[1, rk] = DT/Tau*(-XH[1] + 100*(PSP1)^2/(120^2 + (PSP1)^2));  #Your Equation Here
  	K[2, rk] = DT/Tau*(-XH[2] + 100*(PSP2)^2/(120^2 + (PSP2)^2));    #Your Equation Here
  	K[3, rk] = DT/Tau*(-XH[3] + 100*(PSP3)^2/(120^2 + (PSP3)^2));    #Your Equation Here
  	K[4, rk] = DT/Tau*(-XH[4] + 100*(PSP4)^2/(120^2 + (PSP4)^2));    #Your Equation Here
  	K[5, rk] = DT/Tau*(-XH[5] + 100*(PSP5)^2/(120^2 + (PSP5)^2));    #Your Equation Here

 end
	X[:, T] = X[:, T-1] + sum((Weights.*K)', 1)'/6
end

Xiso = 0:100;  #X for Isoclines

Isocline1 = 100*(Stim1 - 3*Xiso).^2./(120^2 + (Stim1 - 3*Xiso).^2).*(3*Xiso .< Stim1)
Isocline2 = 100*(Stim2 - 3*Xiso).^2./(120^2 + (Stim2 - 3*Xiso).^2).*(3*Xiso .< Stim2)

trace1 = PlotlyJS.scatter(;x=Xiso, y=Isocline1, mode="lines", line_color="red")
trace2 = PlotlyJS.scatter(;x=Xiso, y=Isocline2, mode="lines", line_color="blue")
layout = PlotlyJS.Layout(title="Isoclines")
p1 = PlotlyJS.plot([trace1, trace2], layout)

# figure(1); Za = plot(Time, X); set(Za, "LineWidth", 2)

Peak = maximum(X[1, :])
Thresh = X[1, :] .> 0.95*Peak
[Y, Tm] = maximum(Thresh)
Y = maximum(Thresh)
Tm = find()
Latency = Time[Tm]


LoadError: [91msyntax: invalid assignment location "[Y,Tm]"[39m